In [ ]:
import tensorflow as tf
slim = tf.contrib.slim
import numpy as np
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
# load and split train validation set
X = np.load('../data/features/data_x.npy')
Y = np.load('../data/features/data_y.npy')

tr_features, tr_labels = shuffle(X, Y, random_state=0)
a = tr_features.shape[0]

tr_features = tr_features[:int(a*0.3)]
tr_labels = tr_labels[:int(a*0.3)]


train_features, test_features, train_labels, test_labels = train_test_split(tr_features,
                                                                            tr_labels, 
                                                                            test_size = 0.3, 
                                                                            random_state = 42)

print('Train shape: ', train_features.shape)
print('Test shape: ', test_features.shape)

# CNN+RNN arcitecture from 

https://www.isca-speech.org/archive/Interspeech_2017/pdfs/0360.PDF

In [ ]:
# 
# Network Parameters
n_input = 20 
n_steps = 30
n_hidden = 300
n_classes = 2


############ Conv-1 ###############
net = slim.conv2d(train_features, 16, [5, 5], stride=[1, 1], scope='conv1')
net = PReLU(net, 'conv1_activation')
net = tf.nn.max_pool2d(net, strides=[2,1], ksize=[2,2], padding='VALID', name='pool1')

############ Conv-2 ###############
net = slim.conv2d(net, 16, [1, 1], stride=[1, 1], scope='conv2a')
net = PReLU(net, 'conv2a_activation')
net = slim.conv2d(net, 32, [1, 1], stride=[1, 1], scope='conv2b')
net = PReLU(net, 'conv2b_activation')
net = tf.nn.max_pool2d(net, strides=[2,1], ksize=[2,2], padding='VALID', name='pool2')

############ Conv-3 ###############
net = slim.conv2d(net, 32, [1, 1], stride=[1, 1], scope='conv3a')
net = PReLU(net, 'conv3a_activation')
net = slim.conv2d(net, 16, [3, 3], stride=[1, 1], scope='conv3b')
net = PReLU(net, 'conv3b_activation')
net = tf.nn.max_pool2d(net, strides=[2,1], ksize=[2,2], padding='VALID', name='pool3')

############ BGRU ###############
def BGRU(x, weight, bias, n_steps, n_hidden):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)
    
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, n_steps, 1)
    
    # Define GRU cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = tf.nn.rnn_cell.GRUCell(n_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = tf.nn.rnn_cell.GRUCell(n_hidden, forget_bias=1.0)

    # Get BiRNN cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.nn.softmax(tf.matmul(outputs[-1], weight) + bias)
    
weight = tf.Variable(tf.random_normal([n_hidden, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))
prediction = BGRU(net, weight, bias, n_steps, n_hidden)

########## cost funstion and optimizer #############
cost_function = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#
init = tf.global_variables_initializer()

In [ ]:
# tensorboard
session = tf.Session()
#to create the graph summary
writer = tf.summary.FileWriter('logdir', session.graph)

In [ ]:
cost_history = np.empty(shape=[1],dtype=float)
y_true, y_pred = None, None
with tf.Session() as session:
    session.run(init)
    
    for epoch in range(training_epochs):            
        _,cost = session.run([optimizer,cost_function],feed_dict={X:train_features,Y:train_labels})
        cost_history = np.append(cost_history,cost)
    
    y_pred = session.run(tf.argmax(y_,1),feed_dict={X: test_features})
    y_true = session.run(tf.argmax(test_labels,1))
    print("Test accuracy: ",
          round(session.run(accuracy, feed_dict={X: test_features,Y: test_labels}),3))